# Перезапускает едро
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [37]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import random as r
from matplotlib.patches import Ellipse, Arc

from tqdm import tqdm

import math
from time import gmtime, strftime, localtime

In [2]:
class MathDot:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "Dot - (" + str(self.x) + ", " + str(self.y) + ")\n"
        
    def dX(dotOne, dotTwo):
        return dotTwo.x - dotOne.x
    
    def dY(dotOne, dotTwo):
        return dotTwo.y - dotOne.y
    
    def length(dotOne, dotTwo):
        return math.sqrt(pow(MathDot.dY(dotOne, dotTwo), 2) + pow(MathDot.dX(dotOne, dotTwo), 2))
    
    def makeVector(dotOne, dotTwo):
        return MathDot(dotTwo.x - dotOne.x, dotTwo.y - dotOne.y)
    
    def vectorProduct(dotOne, dotTwo):
        return dotOne.x * dotTwo.y - dotOne.y * dotTwo.x
    
class Dot(MathDot):
    
    def __init__(self, x, y, parentBranch):
        self.isBusy = False
        super().__init__(x, y)
        self.parentBranch = parentBranch
        self.childBranch = None
        
    def addChildBranch(self, childBranch):
        self.childBranch = childBranch

In [3]:
class Direction:
    
    def __init__(self, degree):
        self.degree = degree
        self.sin = round(math.sin(math.radians(degree)),3)
        self.cos = round(math.cos(math.radians(degree)),3)
        
    def __add__(self, other):
        if isinstance(other, Direction):
            return Direction(self.degree + other.degree)
        else:
            return Direction(self.degree + other)
        
    def __sub__(self, other):
        if isinstance(other, Direction):
            return Direction(self.degree - other.degree)
        else:
            return Direction(self.degree - other)

In [4]:
class Branch:
    
    def __init__(self, xStart, yStart, direction, fatherBranch = None):
        self.startCoordinates = (xStart, yStart)
        self.endCoordinates = [xStart, yStart]
        self.direction = direction
        self.dots = [Dot(xStart, yStart, self)]
        self.freeDots = []
        self.childBranches = []
        self.fatherBranch = fatherBranch
        self.canGrow = True
        self.outsideGrowArea = False
        self.intersectedBranches = [[]]
        
    def __str__(self):
        string = "Branch:\nFirst dot - (" + str(self.dots[0].x) + ", " + str(self.dots[0].y) + ")\n" + \
        "Last dot - (" + str(self.dots[-1].x) + ", " + str(self.dots[-1].y) + ")\nDirection - " + str(self.direction.degree) + "\n"
        return string
        
    def addDot(self):
        self.endCoordinates[0] += self.direction.cos * CONSTS.MIN_INCREASE_LENGTH
        self.endCoordinates[1] += self.direction.sin * CONSTS.MIN_INCREASE_LENGTH
        newDot = Dot(self.endCoordinates[0], self.endCoordinates[1], self)
        if self.dots[-1].isBusy == False:
            self.freeDots.append(self.dots[-1])
        self.dots.append(newDot)
        
        self.intersectedBranches.append([])
        
        return newDot
    
    def removeDot(self):
        for branch in self.intersectedBranches[-1]:
            branch.canGrow = not branch.outsideGrowArea
        self.intersectedBranches.pop()
        self.dots.pop()
        if len(self.freeDots) > 0 and self.dots[-1] == self.freeDots[-1]:
            self.freeDots.pop()
        self.endCoordinates[0] = self.dots[-1].x
        self.endCoordinates[1] = self.dots[-1].y
        self.canGrow = not self.outsideGrowArea
        
    def addChildBranch(self, dot, coef):
        coef = 1 if coef >= 0 else -1
        newDirection = Direction(self.direction.degree + coef * CONSTS.NEW_BRANCH_ANGLE_DEGREE)
        thisDot = dot
        nextDot = self.dots[self.dots.index(dot) + 1]
        thisDot.isBusy = True
        nextDot.isBusy = True
        if thisDot in self.freeDots:
            self.freeDots.remove(thisDot)
        if nextDot in self.freeDots:
            self.freeDots.remove(nextDot)
        newBranchXStart = (thisDot.x + nextDot.x) / 2
        newBranchYStart = (thisDot.y + nextDot.y) / 2
        newBranch = Branch(newBranchXStart, newBranchYStart, newDirection, self)
        thisDot.childBranch = newBranch
        nextDot.childBranch = newBranch
        self.childBranches.append(newBranch)
        return newBranch
        
    def getFreeDots(self):
        return self.freeDots
    
    def addIntersection(self, intersectionDot, branch):
        fromStartToIntersectionLength = MathDot.length(self.dots[0], intersectionDot)
        intersectedDotIndex = len(self.dots) - 1 - (self.length() - fromStartToIntersectionLength) // CONSTS.MIN_INCREASE_LENGTH
        intersectedDotIndex = int(intersectedDotIndex)
        if intersectedDotIndex == len(self.dots):
            intersectedDotIndex -= 1
        self.intersectedBranches[intersectedDotIndex].append(branch)        
        
    def dX(self):
        return self.endCoordinates[0] - self.startCoordinates[0]
    
    def dY(self):
        return self.endCoordinates[1] - self.startCoordinates[1];
    
    def length(self):
        return math.sqrt(pow(self.dX(), 2) + pow(self.dY(), 2))
    
    def getXCoordinates(self):
        return [self.startCoordinates[0], self.endCoordinates[0]]

    def getYCoordinates(self):
        return [self.startCoordinates[1], self.endCoordinates[1]]

In [5]:
class Tree:
    
    def __init__(self, root):
        self.root = root
        self.allBranches = [root]
    
    def getAllBranches(self):
        return self.allBranches
        
    def _recGetAllBranches(self, branch, res):
        res.append(branch)
        for b in branch.childBranches:
            self._recGetAllBranches(b, res)
    
    def getFreeDots(self):
        res = []
        for branch in self.getAllBranches():
            for dot in branch.getFreeDots():
                res.append(dot)
        return res
    
    def addChildBranch(self, dot, coef):
        self.allBranches.append(dot.parentBranch.addChildBranch(dot, coef))    

In [164]:
class Cell:

    def __init__(self, initCoords : list, initRad : int):
        self.centre = Dot(initCoords[0], initCoords[1], None)
        self.startRadius = initRad
        self.radius = initRad
        self.leanedBranches = []
        
    def grow(self):
        for lBranch in self.leanedBranches:
            lBranch.canGrow = True
        self.leanedBranches.clear()
        self.radius += CONSTS.CELL_RADIUS_PLUS
    
    def addLeanedBranch(self, branch):
        self.leanedBranches.append(branch)
        branch.canGrow = False
        branch.outsideGrowArea = False

In [173]:
class Plot:
    
    def __init__(self, x_limits : list, y_limits : list):
        self.x_limits = x_limits
        self.y_limits = y_limits
        
    def drawPlot(self, tree : Tree, cell : Cell, iter_count : int, name : str = None):
        self.fig, self.ax = plt.subplots()
        self.ax.set_xlim(self.x_limits)
        self.ax.set_ylim(self.y_limits)
        self.__drawTree(tree)
        self.__drawCell(cell)
        self.__drawText(cell, tree, iter_count)
        self.__savePlot(name)
    
    def __savePlot(self, name : str):
        self.fig.set_size_inches(10,20)
        plt.savefig("img/scr_" + name + ".png")
        plt.close('all')
    
    def __drawTree(self, tree : Tree):
        for branch in tree.getAllBranches():
            self.ax.plot(branch.getXCoordinates(), branch.getYCoordinates(), c = 'r')
            
    def __drawCell(self, cell : Cell):
        cellWall = Arc(xy = [cell.centre.x, cell.centre.y], width = cell.radius * 2, height = cell.radius * 2,theta1 = -90, theta2 = 90, color = "g")
        cellAttachArea = Arc(xy = [cell.centre.x, cell.centre.y], width = (cell.radius - CONSTS.DISTANCE_TO_ATTACH) * 2, height = (cell.radius - CONSTS.DISTANCE_TO_ATTACH) * 2, theta1 = -90, theta2 = 90, color = "g", ls = '--')
        cellGrowArea = Arc(xy = [cell.centre.x, cell.centre.y], width = (cell.radius - CONSTS.DISTANCE_TO_GROW) * 2, height = (cell.radius - CONSTS.DISTANCE_TO_GROW) * 2, theta1 = -90, theta2 = 90, color = "g", ls = '-')
        
        self.ax.add_patch(cellWall) 
        self.ax.add_patch(cellAttachArea) 
        self.ax.add_patch(cellGrowArea)
        
        leanedDots = [x.dots[-1] for x in cell.leanedBranches]
        circles = [plt.Circle((dot.x, dot.y), 0.3, color = 'b') for dot in leanedDots]
        for circle in circles:
            self.ax.add_patch(circle)
    
    def __drawText(self, cell : Cell, tree : Tree, iter_count : int):
        text = ""
        text += "\nn: " + str(CONSTS.N_BRANCHES_TO_PUSH_SELL)
        text += "\nKb: " + str(CONSTS.Kb)
        text += "\nH: " + str(CONSTS.DISTANCE_TO_GROW)
        text += "\nD: " + str(CONSTS.DISTANCE_TO_ATTACH)
        text += "\navgV: " + str(round((cell.radius - cell.startRadius) * 5.5 / 10 / (iter_count * CONSTS.DT), 2)) + "e-8 м/с"
        self.ax.annotate(text, xy=(self.x_limits[1] - 40, self.y_limits[1] - 40), color = 'r')

In [147]:
class Checker:
    
    def __init__(self, tree, cell):
        self.tree = tree
        self.cell = cell
        
    def allBranchesThatCanGrow(self):
        return [branch for branch in self.tree.getAllBranches() if branch.canGrow and self.checkBranchCanGrow(branch)]
    
    def allBranchesThatCanBreak(self):
        return [branch for branch in self.tree.getAllBranches() if self.checkBranchCanBreak(branch)]
          
    def allBranchesInsideGrowArea(self):
        return [branch for branch in self.tree.getAllBranches() if branch.outsideGrowArea == False]
    
    def checkBranchOutsideGrowArea(self, branch : Branch):
        length = MathDot.length(self.cell.centre, branch.dots[-1])
        return abs(length - self.cell.radius) >= 1e-10 \
            and abs(length - (self.cell.radius - CONSTS.DISTANCE_TO_GROW if self.cell.radius > CONSTS.DISTANCE_TO_GROW else 0)) >= 1e-10 \
            and (length > self.cell.radius \
            or length < self.cell.radius - CONSTS.DISTANCE_TO_GROW)
    
    def checkBranchCanGrow(self, branch):
        for anotherBranch in self.allBranchesInsideGrowArea():
            if anotherBranch != branch and anotherBranch != branch.fatherBranch and anotherBranch not in branch.childBranches:
                res = self.checkGrowBranchCrossAnother(branch, anotherBranch)
                if res != False:
                    branch.canGrow = False
                    if res != True:
                        anotherBranch.addIntersection(res, branch)
                    return False
        return True
    
    def checkBranchCanBreak(self, branch):
        return branch.dots[-1].isBusy == False and len(branch.dots) > 2
    
    def checkGrowBranchCrossAnother(self, branchToGrow, branchTwo):
        
        a = branchToGrow.dots[0]
        lastDot = branchToGrow.dots[-1]
        b = MathDot(lastDot.x + branchToGrow.direction.cos * CONSTS.MIN_INCREASE_LENGTH, lastDot.y + branchToGrow.direction.sin * CONSTS.MIN_INCREASE_LENGTH)
        c = branchTwo.dots[0]
        d = branchTwo.dots[-1]
        
        return self.__intersection(a,b,c,d)
        
    def __intersection(self, a,b,c,d):
        x1 = a.x
        y1 = a.y
        x2 = b.x
        y2 = b.y
        x3 = c.x
        y3 = c.y
        x4 = d.x
        y4 = d.y

        A1 = y1 - y2
        A2 = y3 - y4
        B1 = x2 - x1
        B2 = x4 - x3
        C1 = x1 * y2 - x2 * y1
        C2 = x3 * y4 - x4 * y3
        
        if abs(c.x - d.x) <= 1e-10:
            return self.__dotIsOnSegment(c, a, b)

        if abs(A1) <= 1e-10 and abs(A2) <= 1e-10:
            if C1 / B1 == C2 / B2:
                m = max(x1, x2, x3, x4)
                if m == x1 or m == x2:
                    m = min(x1, x2)
                    return self.__dotIsOnSegment(a if m == x1 else b, c, d)
                else:
                    m = min(x3, x4)
                    return self.__dotIsOnSegment(c if m == x3 else d, a, b)
            else:
                return False

        if abs(A1) <= 1e-10:
            y = -C1 / B1
            x = -C2 / A2 + -B2 / A2 * y
            if self.__dotIsInTwoSegments(MathDot(x,y), a, b, c, d):
                return MathDot(x,y)
            else:
                return False

        if abs(A2) <= 1e-10:
            y = -C2 / B2
            x = -C1 / A1 + -B1 / A1 * y
            if self.__dotIsInTwoSegments(MathDot(x,y), a, b, c, d):
                return MathDot(x,y)
            else:
                return False

        value = B1 / A1 - B2 / A2
        if abs(value) <= 1e-10:
            return False

        y = (C2 / A2 - C1 / A1) / value
        x = -C1 / A1 + -B1 / A1 * y
        if self.__dotIsInTwoSegments(MathDot(x,y), a, b, c, d):
            return MathDot(x,y)
        else:
            return False
    
    def __dotIsInTwoSegments(self, dot, a, b, c, d):
        return self.__dotIsOnSegment(dot, a, b) and self.__dotIsOnSegment(dot, c, d)

    def __dotIsOnSegment(self, dot, a, b):
        #print(dot, a, b)
        x = dot.x
        y = dot.y
        x1 = a.x
        y1 = a.y
        x2 = b.x
        y2 = b.y
    
        return abs((x - x1) * (y2 - y1) - (y - y1) * (x2 - x1)) <= 1e-10 and (x1 <= x <= x2 or x2 <= x <= x1)
        
    def checkBranchCanHaveChildBranches(self, branch, coef):
        coef = 1 if coef >= 0 else -1
        newAngle = branch.direction.degree + coef * CONSTS.NEW_BRANCH_ANGLE_DEGREE
        if newAngle > CONSTS.MAX_BRANCH_ANGLE or newAngle < CONSTS.MIN_BRANCH_ANGLE:
            return False
        return True
    
    
    def checkDotCanJoin(self, dot : Dot):
        if MathDot.length(self.cell.centre, dot) < self.cell.radius and MathDot.length(self.cell.centre, dot) > self.cell.radius - CONSTS.DISTANCE_TO_ATTACH:
            return True
        return False
    
    def checkDotOutsideCell(self, dot : Dot):
        if abs(MathDot.length(self.cell.centre, dot) - self.cell.radius) >= 1e-10 and MathDot.length(self.cell.centre, dot) > self.cell.radius:
            return True
        return False
    
    def checkCellNeedToGrow(self):
        return len(self.cell.leanedBranches) >= CONSTS.N_BRANCHES_TO_PUSH_SELL

In [94]:
a = 1 < 2 or \
5 > 6
print(a)

True


In [9]:
class Error:
    value = 0

In [165]:
class Randomizer:

    def __init__(self, tree : Tree, cell : Cell):
        self.tree = tree
        self.cell = cell
        self.checker = Checker(tree, cell)
    
    def oneTick(self):
        self.grow()
        self.breakBranches()
        self.addChildBranches()
        if self.checker.checkCellNeedToGrow():
            self.cellGrow()
       # if len(myRandomizer.checker.allBranchesInsideGrowArea()) != len([x for x in myTree.getAllBranches() if myRandomizer.checker.checkBranchOutsideGrowArea(x) == False]):
           # raise Exception("seva")
                
    def grow(self):
        for branch in self.checker.allBranchesThatCanGrow():
            if r.random() < CONSTS.ADD_DOT_PROB:
                newDot = branch.addDot()
                if self.checker.checkDotOutsideCell(newDot):
                    self.cell.addLeanedBranch(newDot.parentBranch)
    
                
    def addChildBranches(self):
        allDots = self.tree.getFreeDots()
        for dot in allDots:
            if dot.isBusy == False and self.checker.checkDotCanJoin(dot) and r.random() < CONSTS.CREATE_BRANCH_PROB:
                if self.checker.checkBranchCanHaveChildBranches(dot.parentBranch, 1) == False:
                    coef = -1
                elif self.checker.checkBranchCanHaveChildBranches(dot.parentBranch, -1) == False:
                    coef = 1
                else:
                    coef = 1 if r.random() >= 0.5 else -1
                self.tree.addChildBranch(dot, coef)
                
    def breakBranches(self):
        for branch in self.checker.allBranchesThatCanBreak():
            if r.random() < CONSTS.REMOVE_DOT_PROB:
                branch.removeDot()
                if branch in self.cell.leanedBranches:
                    self.cell.leanedBranches.remove(branch)
                    
    def cellGrow(self):
        self.cell.grow()
        for branch in self.checker.allBranchesInsideGrowArea():
            if self.checker.checkBranchOutsideGrowArea(branch) == True:
                branch.outsideGrowArea = not self.checker.checkDotOutsideCell(branch.dots[-1])
                if branch.outsideGrowArea:
                    branch.canGrow = False

In [188]:
class CONSTS: 
    DT = 0.1
    K1G = 10
    K2 = 1.4
    K3ARP = 10
    X_DENSITY = 9.7e-6
    Kb = 20000
    ADD_DOT_PROB = 1-1/(math.exp(DT*K1G))
    REMOVE_DOT_PROB = 1-1/(math.exp(DT*K2))
    CREATE_BRANCH_PROB = 1-1/(math.exp(DT*Kb*X_DENSITY))
    #ADD_DOT_PROB = 0.5
    #REMOVE_DOT_PROB = 0.2
    #CREATE_BRANCH_PROB = 0.9
    N_BRANCHES_TO_PUSH_SELL = 4
    NEW_BRANCH_ANGLE_DEGREE = 70
    MIN_INCREASE_LENGTH = 1
    MAX_BRANCH_ANGLE = 90
    MIN_BRANCH_ANGLE = -90
    DISTANCE_TO_ATTACH = 5
    DISTANCE_TO_GROW = 10
    CELL_RADIUS_PLUS = 1

In [189]:
print("ADD_DOT_PROB" , CONSTS.ADD_DOT_PROB)
print("REMOVE_DOT_PROB" , CONSTS.REMOVE_DOT_PROB)
print("CREATE_BRANCH_PROB" , CONSTS.CREATE_BRANCH_PROB)

ADD_DOT_PROB 0.6321205588285577
REMOVE_DOT_PROB 0.13064176460119403
CREATE_BRANCH_PROB 0.019213031018206927


In [191]:
myTree = Tree(Branch(0,0,Direction(0),None))
myCell = Cell([0,0], 10)
myRandomizer = Randomizer(myTree, myCell)
#myPlot = Plot([0, 200], [-200, 200])
iter_count = 1500
for i in tqdm(range(iter_count)):
    myRandomizer.oneTick()
    index += 1
    #plot.drawPlot(tree, cell, index)
myPlot = Plot([0, myCell.radius + 20], [-(myCell.radius + 20), myCell.radius + 20])
myPlot.drawPlot(myTree, myCell, name = strftime("%d-%m-%Y_%H_%M", localtime()), iter_count = iter_count)

100%|██████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:39<00:00, 37.94it/s]


In [184]:
print(myCell.radius)

172


In [186]:
myRandomizer.cellGrow()

In [187]:
print(len(myRandomizer.checker.allBranchesInsideGrowArea()))
print(len([x for x in myTree.getAllBranches() if myRandomizer.checker.checkBranchOutsideGrowArea(x) == False]))
print(len(myRandomizer.checker.allBranchesThatCanGrow()))

79
79
61


In [170]:
a = myRandomizer.checker.allBranchesInsideGrowArea()
b = [x for x in myTree.getAllBranches() if myRandomizer.checker.checkBranchOutsideGrowArea(x) == False]
for x in a:
    if x not in b:
        print(1,x)
        c = x

for x in b:
    if x not in a:
        print(2,x)
print(c.outsideGrowArea)
print(myRandomizer.checker.checkBranchOutsideGrowArea(c))
print(myRandomizer.checker.checkDotOutsideCell(c.dots[-1]))

False
True
False


In [171]:
print(myCell.leanedBranches[0] == c)

IndexError: list index out of range

In [137]:
print(MathDot.length(myCell.centre, c.dots[-1]) > myCell.radius)
print(MathDot.length(myCell.centre, c.dots[-1]) < myCell.radius - CONSTS.DISTANCE_TO_GROW)

False
False


In [178]:
print(len(myTree.getAllBranches()))
res = []
myTree._recGetAllBranches(myTree.root, res)
print(len(res))
print(len([x for x in res if x.canGrow]))

7182
7182
74


In [212]:
print(len([x for x in tree.getAllBranches() if x.canGrow]))

6


In [152]:
(cell.radius - cell.startRadius) * 5.5e-9 / 40

NameError: name 'cell' is not defined

In [13]:
a = [1,2,3]
a.remove(5)

ValueError: list.remove(x): x not in list

In [108]:
print(format(44.553423e-8, '.3g'))

4.46e-07


In [164]:
a = True
b = a == False
print(b)

False
